In [ ]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2021-03-19 14:56:47--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  39.2MB/s    in 7.7s    

2021-03-19 14:56:55 (22.6 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2021-03-19 14:56:55--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  15.7MB/s    in 4.9s    

2021-03-19 14:57:00 (12.5 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [ ]:
import keras
from keras.layers.normalization import BatchNormalization
import numpy as np
import scipy.io as io
from keras.layers import Activation, Flatten,Softmax
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.models import Sequential
from keras import optimizers
from keras.backend import function
import pandas as pd
import tensorflow as tf

In [ ]:
train_set = io.loadmat("train_32x32.mat")
test_set = io.loadmat("test_32x32.mat")
x_train = train_set['X']
y_train = train_set['y']
x_test = test_set['X']
y_test = test_set['y']
# Reshaping test and training data sets
x_train = np.moveaxis(x_train,[3,0,1,2],[0,1,2,3])
x_train = x_train.astype('float32')
x_test = np.moveaxis(x_test,[3,0,1,2],[0,1,2,3])
x_test = np.asarray(x_test)
x_test = x_test.astype('float32')
#Normalisation des données
x_train /= x_train.max()
x_test /= x_test.max()
# Passage de classe comme integer à classe comme vecteur de dimension classes
y_test_RN = [y-1 for y in y_test]# la classe 1 corresponds au array qui à 1 en indice 0
y_train_RN = [y-1 for y in y_train]
classes = 10
y_train_RN = keras.utils.to_categorical(y_train_RN, classes)
y_test_RN = keras.utils.to_categorical(y_test_RN, classes)
 #Définition du type de data pour optimiser la mémoire
y_train_RN = y_train_RN.astype('int32')
y_test_RN = y_test_RN.astype('int32')
# Featurewise normalization
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=False,
    zca_whitening=False, 
    zca_epsilon=1e-06)

datagen.fit(x_train)

In [ ]:
y_test_KNN=[k.item() for k in y_test]
y_train_KNN=[k.item() for k in y_train]

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Projet 3A')

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
S1 (MaxPooling2D)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
S2 (MaxPooling2D)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 5, 5, 128)         147584    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 3, 128)         147584    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 1, 128)        

In [ ]:
x_train_10000=x_train[:10000]

In [ ]:
def takeSecond(elem):
    return elem[1]

In [ ]:
def somme_array(L):
  somme=np.zeros(len(L[0]))
  for l in L:
    somme=somme+l
  return somme

In [ ]:
def neighbors(x,d,k):
  distance=[]
  voisin=[]
  for i in range(len(d)):
    L=[i]
    L.append(np.linalg.norm(x-d[i]))
    distance.append(L)
  distance=sorted(distance,key=takeSecond)
  for j in range(k):
    voisin.append(distance[j][0])
  return voisin

In [ ]:
def predict(i,t1,t2,d1,d2,k,model):
  voisins1=neighbors(t1[i],d1,k)
  voisins2=neighbors(t2[i],d2,k)
  neig=[]
  for N in voisins1:
    neig.append(model.predict(x_train[N:N+1])[0])
  for N in voisins2:
    neig.append(model.predict(x_train[N:N+1])[0])
  v=somme_array(neig)
  classe=np.argmax(v, axis=0)+1
  return classe 

# Test somme

In [ ]:
get_conv50_output = function([model.layers[0].input],
                                  [model.layers[6].output])
layer_output_conv50 = get_conv50_output([x_train_10000])

In [ ]:
conv50=np.array(layer_output_conv50[0]).reshape(10000,128)

In [ ]:
layer_output_conv50_test = get_conv50_output([x_test[:5000]])

In [ ]:
conv50_test=np.array(layer_output_conv50_test[0]).reshape(5000,128)

In [ ]:
get_conv40_output = function([model.layers[0].input],
                                  [model.layers[5].output])
layer_output_conv40 = get_conv40_output([x_train_10000])

In [ ]:
conv40=np.array(layer_output_conv40[0]).reshape(10000,1152)

In [ ]:
layer_output_conv40_test = get_conv40_output([x_test[:5000]])

In [ ]:
conv40_test=np.array(layer_output_conv40_test[0]).reshape(5000,1152)

In [ ]:
prediction=[]
for k in range(5000):
  prediction.append(predict(k,conv40_test,conv50_test,conv40,conv50,5,model))
prediction

[5,
 2,
 1,
 10,
 6,
 1,
 9,
 1,
 1,
 8,
 3,
 8,
 5,
 1,
 4,
 4,
 1,
 6,
 3,
 4,
 2,
 10,
 1,
 3,
 2,
 5,
 4,
 1,
 4,
 2,
 9,
 3,
 8,
 6,
 10,
 1,
 5,
 1,
 1,
 2,
 9,
 1,
 6,
 9,
 9,
 8,
 1,
 2,
 10,
 6,
 9,
 1,
 5,
 1,
 9,
 2,
 4,
 5,
 1,
 5,
 2,
 9,
 2,
 4,
 1,
 10,
 6,
 9,
 3,
 2,
 3,
 5,
 1,
 1,
 4,
 6,
 2,
 6,
 3,
 1,
 4,
 1,
 7,
 1,
 2,
 1,
 7,
 1,
 10,
 1,
 4,
 4,
 6,
 4,
 4,
 2,
 7,
 1,
 7,
 7,
 4,
 8,
 2,
 3,
 2,
 2,
 3,
 1,
 8,
 6,
 3,
 7,
 6,
 3,
 10,
 7,
 2,
 7,
 4,
 2,
 8,
 8,
 1,
 7,
 4,
 9,
 1,
 4,
 2,
 9,
 6,
 1,
 6,
 10,
 8,
 2,
 10,
 1,
 10,
 3,
 1,
 2,
 4,
 6,
 7,
 2,
 1,
 3,
 5,
 3,
 1,
 5,
 9,
 1,
 8,
 1,
 8,
 6,
 4,
 6,
 4,
 9,
 3,
 6,
 1,
 10,
 1,
 2,
 6,
 2,
 7,
 5,
 2,
 2,
 1,
 4,
 2,
 9,
 6,
 7,
 1,
 10,
 2,
 1,
 4,
 8,
 3,
 2,
 1,
 6,
 2,
 6,
 1,
 7,
 2,
 3,
 6,
 1,
 1,
 3,
 7,
 2,
 3,
 5,
 10,
 2,
 1,
 10,
 4,
 7,
 1,
 10,
 2,
 3,
 7,
 3,
 1,
 1,
 1,
 2,
 10,
 4,
 7,
 1,
 2,
 3,
 6,
 2,
 2,
 9,
 2,
 5,
 7,
 4,
 1,
 1,
 3,
 4,
 3,
 3,
 7,
 1,
 1,
 4,
 1,
 7,


In [ ]:
right=0
for k in range(5000):
  if prediction[k]==y_test_KNN[k]:
    right=right+1
right


4587

In [ ]:
score=right/5000
score


0.9174

# Test 2

In [ ]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    if lst3==[]:
      return lst2
    return lst3 

In [ ]:
neighbors(conv40_test[1],conv40,5)

[6963, 6701, 1153, 2118, 6144]

In [ ]:
def predict1(i,t1,t2,d1,d2,k,model):
  voisins1=neighbors(t1[i],d1,k)
  voisins2=neighbors(t2[i],d2,k)
  voisins=intersection(voisins1,voisins2)
  neig=[]
  for N in voisins:
    neig.append(model.predict(x_train[N:N+1])[0])
  v=somme_array(neig)
  classe=np.argmax(v, axis=0)+1
  return classe 

In [ ]:
prediction=[]
for k in range(5000):
  prediction.append(predict1(k,conv40_test,conv50_test,conv40,conv50,5,model))
prediction

[5,
 2,
 1,
 10,
 6,
 1,
 9,
 1,
 1,
 8,
 3,
 6,
 5,
 1,
 4,
 4,
 1,
 6,
 3,
 4,
 2,
 10,
 1,
 9,
 2,
 5,
 4,
 1,
 4,
 2,
 9,
 3,
 8,
 6,
 10,
 1,
 5,
 1,
 1,
 2,
 8,
 1,
 6,
 9,
 9,
 8,
 1,
 2,
 10,
 6,
 9,
 1,
 5,
 1,
 9,
 2,
 4,
 5,
 1,
 9,
 2,
 9,
 2,
 4,
 1,
 10,
 6,
 9,
 3,
 2,
 3,
 5,
 1,
 1,
 4,
 6,
 2,
 6,
 3,
 1,
 4,
 1,
 7,
 1,
 2,
 1,
 7,
 1,
 10,
 1,
 4,
 4,
 6,
 4,
 4,
 2,
 7,
 1,
 7,
 7,
 4,
 8,
 2,
 3,
 2,
 2,
 3,
 1,
 8,
 6,
 3,
 7,
 6,
 5,
 10,
 7,
 2,
 7,
 4,
 2,
 8,
 8,
 1,
 7,
 4,
 9,
 1,
 4,
 2,
 9,
 6,
 1,
 8,
 10,
 8,
 2,
 10,
 1,
 10,
 3,
 1,
 2,
 4,
 6,
 7,
 2,
 1,
 3,
 5,
 3,
 1,
 5,
 9,
 1,
 8,
 1,
 3,
 6,
 4,
 6,
 4,
 9,
 3,
 6,
 1,
 10,
 1,
 2,
 6,
 2,
 7,
 9,
 2,
 2,
 1,
 4,
 2,
 9,
 6,
 7,
 1,
 10,
 2,
 1,
 4,
 8,
 3,
 2,
 1,
 6,
 2,
 6,
 1,
 7,
 2,
 3,
 6,
 1,
 1,
 3,
 7,
 2,
 3,
 5,
 10,
 2,
 1,
 10,
 4,
 7,
 1,
 10,
 2,
 3,
 7,
 3,
 1,
 1,
 1,
 2,
 10,
 10,
 7,
 1,
 2,
 3,
 6,
 2,
 2,
 9,
 2,
 5,
 7,
 4,
 1,
 1,
 3,
 4,
 3,
 2,
 7,
 1,
 1,
 4,
 1,
 7,

In [ ]:
right=0
for k in range(5000):
  if prediction[k]==y_test_KNN[k]:
    right=right+1
right

4584

In [ ]:
score=right/5000
score

0.9168